In [4]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "CH01-Basic"
os.environ["LANGSMITH_PROJECT"] = project_name

### 데이터를 효과적으로 전달하는 방법

- `RunnablePassthrough`는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있다.
- `RunnablePassthrough()`가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달한다.
- `RunnablePassthrough.assign(...)`방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가한다.

#### RunnablePassthrough

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# prompt 와 llm 을 생성한다
prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0)

# chain을 생성한다.
chain = prompt | llm

chain을 `invoke()`하여 실행할 때는 입력 데이터의 타입이 딕셔너리여야 한다.

In [6]:
# chain 을 실행한다
chain.invoke({"num": 5})

AIMessage(content='50입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CN3A5kQiteTqXkAe4sZT4VTB36dCq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--85398806-941b-4de1-b693-c97f7352122a-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

하지만, langchain 라이브러리가 업데이트 되면서 1개의 변수만 템플릿에 포함하고 있다면, 값만 전달하는 것도 가능하다.

In [7]:
# chain 을 실행
chain.invoke(5)

AIMessage(content='50입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CN3BLnUwrHgWknNf9BdocF6MldDsN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--eca3d9a5-3b5a-4660-8123-17f51760374d-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

아래는 `RunnablePassthrough`를 사용한 예제이다. 

`RunnablePassthrough`는 `runnable` 객체이며, `runnable` 객체는 `invoke()` 메소드를 사용하여 별도 실행이 가능하다

In [8]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

아래는 `RunnablePassthrough` 로 체인을 구성하는 예제이다

In [9]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경되었다
runnable_chain.invoke(10)

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CN3BR5e67BrfJSjoxvj9DN6yPgBec', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--25909153-afb1-47e9-9002-72c742c4c16d-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

다음은 `RunnablePassthrough.assign()` 을 사용하는 경우와 비교한 결과이다

In [10]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

`RunnablePassthrough.assign()`

- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합친다

In [ ]:
# 입력 키: num, 할당(assign) 키:: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

### RunnableParallel

In [16]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성한다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있다.
runnable = RunnableParallel(
    # RunnablePaathrough 인스턴스를 'passed' 키워드 인자로 전달한다. 이는 입력된 데이터를 그대로 통과시키는 역할을 한다
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 Runnablepassthrough.assign을 사용하여, 'mult' 람다 함수를 할당한다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킨다.
    extra=RunnablePassthrough.assign(multi=lambda x:x["num"] *3),
    # 'modified' 키워드 인자로 람다 함수를 전달한다
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출한다
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'multi': 3}, 'modified': 2}

Chain도 RunnableParallel 적용할 수 있다

In [20]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [21]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 19, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CN3Gp8zTypYglF95IGy896SE7KVrC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a0b1ef01-2d5b-4498-b498-a8cc389db630-0', usage_metadata={'input_tokens': 19, 'output_tokens': 15, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 면적은 약 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 20, 'total_tok

### RunnableLambda
RunnableLambda를 사용하여 사용자 정의 함수를 맹핑할 수 있다

In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    print(f"입력받은 변수 a의 값: {a}")
    # print(f"입력받은 n의 값: {a['n']}")
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")

# 오늘 날짜를 출력
get_today(None)

입력받은 변수 a의 값: None


'Oct-05'

In [34]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm을 생성한다
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요." 
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

# chain 을 생성한다
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [36]:
# 출력
print(chain.invoke(3))
# print(chain.invoke({"n": 3}))

입력받은 변수 a의 값: 3
다음은 10월 5일이 생일인 유명인 3명입니다:

1. 카를로스 곤잘레스 (Carlos González) - 1985년 10월 5일  
2. 크리스찬 베일 (Christian Bale) - 1974년 10월 5일  
3. 케이트 윈슬렛 (Kate Winslet) - 1975년 10월 5일


`itemgetter` 를 사용하여 특정 키를 추출한다

In [41]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# 문장의 길이를 반환하는 함수입니다.
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [42]:
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25 = 30입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 22, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CN4R62yklpfTIRxDGsuhZC2SpB4BL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--88dd51d0-e24e-419c-a77e-5bed72ad02e3-0', usage_metadata={'input_tokens': 22, 'output_tokens': 9, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})